In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta, TH

In [2]:
path = pd.DataFrame(glob('sample_data_Jan2019/*'),columns=['location'])
path['datadate'] = path['location'].apply(lambda x: x.split('_')[-1].split('.')[0])
path['datadate'] = path['datadate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
path = path.sort_values(['datadate'])
path.reset_index(drop=True,inplace=True)

In [3]:
trade_log=pd.DataFrame(columns=['Entry_date_time', 'Future_price','Transaction_Type', 'Strike','Type_of_Strike','Days_to_expiry',
          'CE_symbol','CE_entry_price','CE_exit_price','CE_exit_datetime','CE_pnl',
          'PE_symbol','PE_entry_price','PE_exit_price','PE_exit_datetime','PE_pnl', 'Total_pnl'])

In [4]:
def type_of_call(i):
    if i == atm:
        return "ATM"
    elif i==itm:
        return "ITM"
    else:
        return "OTM"

In [5]:
#Function for backtesting ATM
def sell(intraday_data,trade_log,atm,ce_entry_price,pe_entry_price,ce_symbol,pe_symbol):
    #stoploss
    stop_loss= 20/100
    ce_stop_loss=ce_entry_price+ce_entry_price*stop_loss
    pe_stop_loss=pe_entry_price+pe_entry_price*stop_loss
    
    #Target
    target_percentage = 80/100
    ce_target = ce_entry_price - ce_entry_price*target_percentage
    pe_target = pe_entry_price - pe_entry_price*target_percentage
    
    ce_stop_loss_counter=0
    pe_stop_loss_counter=0
    ce_exit_datetime=''
    pe_exit_datetime=''
    ce_exit_price=0
    pe_exit_price=0
    ce_pnl=0
    pe_pnl=0
    pnl=0

    for index, row in intraday_data.iterrows():
        ce_ltp = row['ce_close']
        pe_ltp = row['pe_close']
        #Exit Criteria
        #1. CE & PE didn't hit SL & both reach exit time limit 15:10
        if (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0) & (row['datetime']==exit_datetime):
            ce_pnl=ce_entry_price-ce_ltp
            pe_pnl=pe_entry_price-pe_ltp
            ce_stop_loss_counter=1
            pe_stop_loss_counter=1
            ce_exit_datetime=row['datetime']
            pe_exit_datetime=row['datetime']
            ce_exit_price=ce_ltp
            pe_exit_price=pe_ltp
            intraday_data.loc[index,'ce_pnl']=ce_pnl
            intraday_data.loc[index,'pe_pnl']=pe_pnl
            print('CE & PE did not hit SL, both exit at 15:10')
            pnl=ce_pnl+pe_pnl
            break 
        #2. CE is now hit SL, none were hit SL till now
        elif (ce_ltp>=ce_stop_loss) & (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0):
            ce_pnl=ce_entry_price-ce_stop_loss
            pe_pnl=pe_entry_price-pe_ltp
            ce_stop_loss_counter=1
            ce_exit_datetime=row['datetime']
            ce_exit_price=ce_stop_loss
            intraday_data.loc[index,'ce_pnl']=ce_pnl
            intraday_data.loc[index,'pe_pnl']=pe_pnl
            print('CE SL hit')
            pnl=ce_pnl+pe_pnl

        #3. PE is now hit SL, none were hit SL till now
        elif (pe_ltp>=pe_stop_loss) & (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0):
            ce_pnl=ce_entry_price-ce_ltp
            pe_pnl=pe_entry_price-pe_stop_loss
            pe_stop_loss_counter=1
            pe_exit_datetime=row['datetime']
            pe_exit_price=pe_stop_loss
            intraday_data.loc[index,'ce_pnl']=ce_pnl
            intraday_data.loc[index,'pe_pnl']=pe_pnl
            print('PE SL hit')
            pnl=ce_pnl+pe_pnl
        #4. CE was hit SL, (PE is either hit SL or reach exit time limit 15:10)
        elif (ce_stop_loss_counter==1) & (pe_stop_loss_counter==0):
            if (pe_ltp>=pe_stop_loss) & (row['datetime']<exit_datetime):
                pe_pnl=pe_entry_price-pe_stop_loss
                pe_stop_loss_counter=1
                pe_exit_datetime=row['datetime']
                pe_exit_price=pe_stop_loss
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('PE SL hit')
                pnl=ce_pnl+pe_pnl
                break
            elif (row['datetime']==exit_datetime):
                pe_pnl=pe_entry_price-pe_ltp
                pe_stop_loss_counter=1
                pe_exit_datetime=row['datetime']
                pe_exit_price=pe_ltp
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('PE exit at 15:10')
                pnl=ce_pnl+pe_pnl
                break     
        #5. PE was hit SL, (CE is either hit SL or reach exit time limit 15:10)
        elif (pe_stop_loss_counter==1) & (ce_stop_loss_counter==0):
            if (ce_ltp>=ce_stop_loss) & (row['datetime']<exit_datetime):
                ce_pnl=ce_entry_price-ce_stop_loss
                ce_stop_loss_counter=1
                ce_exit_datetime=row['datetime']
                ce_exit_price=ce_stop_loss
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('CE SL hit')
                pnl=ce_pnl+pe_pnl
                break
            elif (row['datetime']==exit_datetime):
                ce_pnl=ce_entry_price-ce_ltp
                ce_stop_loss_counter=1
                ce_exit_datetime=row['datetime']
                ce_exit_price=ce_ltp
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('CE exit at 15:10')
                pnl=ce_pnl+pe_pnl
                break
        #6. update pnl in normal time
        elif ((ce_stop_loss_counter==0 & pe_stop_loss_counter==0) | (ce_stop_loss_counter==1 & pe_stop_loss_counter==0) | (ce_stop_loss_counter==0 & pe_stop_loss_counter==1) | (ce_stop_loss_counter==1 & pe_stop_loss_counter==1)) or (row['datetime']<exit_dateitme):
            ce_pnl=ce_entry_price-ce_ltp
            pe_pnl=pe_entry_price-pe_ltp
            intraday_data.loc[index,'ce_pnl']=ce_pnl
            intraday_data.loc[index,'pe_pnl']=pe_pnl
            pnl=ce_pnl+pe_pnl
        # Exit if target is achieved
        elif pnl >= target_percentage * capital:
            ce_exit_price = ce_ltp
            pe_exit_price = pe_ltp
            ce_exit_datetime = row['datetime']
            pe_exit_datetime = row['datetime']
            ce_pnl = ce_entry_price - ce_exit_price
            pe_pnl = pe_entry_price - pe_exit_price
            pnl = ce_pnl + pe_pnl
            intraday_data.loc[index, 'ce_pnl'] = ce_pnl
            intraday_data.loc[index, 'pe_pnl'] = pe_pnl
            print('Target achieved. Exiting all trades.')
            break
    new_row = {'Entry_date_time': entry_datetime,
               'Future_price': future_price,
               'Transaction_Type':'Sell',
               'Strike': atm,
               'Type_of_Strike':type_of_call(atm),
               'Days_to_expiry': (nearest_expiry - entry_datetime.date()).days,
               'CE_symbol': ce_symbol,
               'CE_entry_price': ce_entry_price,
               'CE_exit_price': ce_exit_price,
               'CE_exit_datetime': ce_exit_datetime,
               'CE_pnl': ce_pnl,
               'PE_symbol': pe_symbol,
               'PE_entry_price': pe_entry_price,
               'PE_exit_price': pe_exit_price,
               'PE_exit_datetime': pe_exit_datetime,
               'PE_pnl': pe_pnl, 
               'Total_pnl': pnl}

    # Convert the dictionary to a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with the trade_log DataFrame
    return pd.concat([trade_log, new_row_df], ignore_index=True)
    

In [6]:
def buy(intraday_data, trade_log, atm,ce_entry_price,pe_entry_price,ce_symbol,pe_symbol):
    # Stop loss
    stop_loss = 20/100
    ce_stop_loss = ce_entry_price - ce_entry_price * stop_loss
    pe_stop_loss = pe_entry_price - pe_entry_price * stop_loss
    
    # Target
    target_percentage = 80/100
    ce_target = ce_entry_price + ce_entry_price * target_percentage
    pe_target = pe_entry_price + pe_entry_price * target_percentage
    
    ce_stop_loss_counter = 0
    pe_stop_loss_counter = 0
    ce_exit_datetime = ''
    pe_exit_datetime = ''
    ce_exit_price = 0
    pe_exit_price = 0
    ce_pnl = 0
    pe_pnl = 0
    pnl = 0

    for index, row in intraday_data.iterrows():
        ce_ltp = row['ce_close']
        pe_ltp = row['pe_close']
        
        # Exit criteria
        # 1. CE & PE didn't hit SL & both reach exit time limit 15:10
        if (ce_stop_loss_counter == 0) and (pe_stop_loss_counter == 0) and (row['datetime'] == exit_datetime):
            ce_pnl = ce_ltp - ce_entry_price
            pe_pnl = pe_ltp - pe_entry_price
            ce_stop_loss_counter = 1
            pe_stop_loss_counter = 1
            ce_exit_datetime = row['datetime']
            pe_exit_datetime = row['datetime']
            ce_exit_price = ce_ltp
            pe_exit_price = pe_ltp
            intraday_data.loc[index, 'ce_pnl'] = ce_pnl
            intraday_data.loc[index, 'pe_pnl'] = pe_pnl
            print('CE & PE did not hit SL, both exit at 15:10')
            pnl = ce_pnl + pe_pnl
            break 
        
        # 2. CE is now hit SL, none were hit SL till now
        elif (ce_ltp <= ce_stop_loss) and (ce_stop_loss_counter == 0) and (pe_stop_loss_counter == 0):
            ce_pnl = ce_ltp - ce_entry_price
            pe_pnl = pe_entry_price - pe_ltp
            ce_stop_loss_counter = 1
            ce_exit_datetime = row['datetime']
            ce_exit_price = ce_ltp
            intraday_data.loc[index, 'ce_pnl'] = ce_pnl
            intraday_data.loc[index, 'pe_pnl'] = pe_pnl
            print('CE SL hit')
            pnl = ce_pnl + pe_pnl

        # 3. PE is now hit SL, none were hit SL till now
        elif (pe_ltp <= pe_stop_loss) and (ce_stop_loss_counter == 0) and (pe_stop_loss_counter == 0):
            ce_pnl = ce_entry_price - ce_ltp
            pe_pnl = pe_ltp - pe_entry_price
            pe_stop_loss_counter = 1
            pe_exit_datetime = row['datetime']
            pe_exit_price = pe_ltp
            intraday_data.loc[index, 'ce_pnl'] = ce_pnl
            intraday_data.loc[index, 'pe_pnl'] = pe_pnl
            print('PE SL hit')
            pnl = ce_pnl + pe_pnl
            
        # 4. CE was hit SL, (PE is either hit SL or reach exit time limit 15:10)
        elif (ce_stop_loss_counter == 1) and (pe_stop_loss_counter == 0):
            if (pe_ltp <= pe_stop_loss) and (row['datetime'] < exit_datetime):
                pe_pnl = pe_stop_loss - pe_entry_price
                pe_stop_loss_counter = 1
                pe_exit_datetime = row['datetime']
                pe_exit_price = pe_stop_loss
                ce_pnl = ce_exit_price - ce_entry_price
                pe_pnl = pe_exit_price-pe_entry_price
                # intraday_data.loc[index, 'ce_pnl'] = ce_pnl
                # intraday_data.loc[index, 'pe_pnl'] = pe_pnl
                print('PE SL hit')
                pnl = ce_pnl + pe_pnl
                break
            elif (row['datetime'] == exit_datetime):
                pe_pnl = pe_ltp - pe_entry_price
                pe_stop_loss_counter = 1
                pe_exit_datetime = row['datetime']
                pe_exit_price = pe_ltp
                ce_pnl = ce_exit_price - ce_entry_price
                pe_pnl = pe_exit_price-pe_entry_price
                # intraday_data.loc[index, 'ce_pnl'] = ce_pnl
                # intraday_data.loc[index, 'pe_pnl'] = pe_pnl
                print('PE exit at 15:10')
                pnl = ce_pnl + pe_pnl
                break     
        
        # 5. PE was hit SL, (CE is either hit SL or reach exit time limit 15:10)
        elif (pe_stop_loss_counter == 1) and (ce_stop_loss_counter == 0):
            if (ce_ltp <= ce_stop_loss) and (row['datetime'] < exit_datetime):
                ce_pnl = ce_stop_loss - ce_entry_price
                ce_stop_loss_counter = 1
                ce_exit_datetime = row['datetime']
                ce_exit_price = ce_ltp
                ce_pnl = ce_exit_price - ce_entry_price
                pe_pnl = pe_exit_price-pe_entry_price
                # intraday_data.loc[index, 'ce_pnl'] = ce_pnl
                # intraday_data.loc[index, 'pe_pnl'] = pe_pnl
                print('CE SL hit')
                pnl = ce_pnl + pe_pnl
                break
            elif (row['datetime'] == exit_datetime):
                ce_pnl = ce_entry_price - ce_ltp
                ce_stop_loss_counter = 1
                ce_exit_datetime = row['datetime']
                ce_exit_price = ce_ltp
                ce_pnl = ce_exit_price - ce_entry_price
                pe_pnl = pe_exit_price-pe_entry_price
                # intraday_data.loc[index, 'ce_pnl'] = ce_pnl
                # intraday_data.loc[index, 'pe_pnl'] = pe_pnl
                print('CE exit at 15:10')
                pnl = ce_pnl + pe_pnl
                break
        
        # 6. Update pnl in normal time
        elif ((ce_stop_loss_counter == 0 & pe_stop_loss_counter == 0) or 
              (ce_stop_loss_counter == 1 & pe_stop_loss_counter == 0) or 
              (ce_stop_loss_counter == 0 & pe_stop_loss_counter == 1) or 
              (ce_stop_loss_counter == 1 & pe_stop_loss_counter == 1)) or (row['datetime'] < exit_datetime):
            ce_pnl = ce_ltp - ce_entry_price
            pe_pnl = pe_ltp - pe_entry_price
            intraday_data.loc[index, 'ce_pnl'] = ce_pnl
            intraday_data.loc[index, 'pe_pnl'] = pe_pnl
            pnl = ce_pnl + pe_pnl

    
    new_row = {'Entry_date_time': entry_datetime,
               'Future_price': future_price,
               'Strike': atm,
               'Type_of_Strike': type_of_call(atm),
               'Transaction_Type':'Buy',
               'Days_to_expiry': (nearest_expiry - entry_datetime.date()).days,
               'CE_symbol': ce_symbol,
               'CE_entry_price': ce_entry_price,
               'CE_exit_price': ce_exit_price,
               'CE_exit_datetime': ce_exit_datetime,
               'CE_pnl': ce_pnl,
               'PE_symbol': pe_symbol,
               'PE_entry_price': pe_entry_price,
               'PE_exit_price': pe_exit_price,
               'PE_exit_datetime': pe_exit_datetime,
               'PE_pnl': pe_pnl, 
               'Total_pnl': pnl}

    # Convert the dictionary to a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with the trade_log DataFrame
    return pd.concat([trade_log, new_row_df], ignore_index=True)


In [7]:
#Preparing the data for backtesting
def data_pre(future_data,ce_data,pe_data):
    future_data=future_data[['datetime','close']].set_index(['datetime'])
    ce_data = ce_data[['datetime','close']].set_index(['datetime'])
    pe_data = pe_data[['datetime','close']].set_index(['datetime'])
    intraday_data = pd.concat([future_data,ce_data,pe_data],axis=1)
    intraday_data.columns=['future_close','ce_close','pe_close']
    intraday_data = intraday_data.ffill()
    intraday_data.reset_index(inplace=True)
    entry_datetime_index=intraday_data[intraday_data['datetime']==entry_datetime].index[0]
    exit_datetime_index=intraday_data[intraday_data['datetime']==exit_datetime].index[0]
    intraday_data=intraday_data[entry_datetime_index:exit_datetime_index+1]
    intraday_data['atm_ce_atm_pnl']=0
    intraday_data['atm_pe_atm_pnl']=0
    intraday_data
    intraday_data.reset_index(drop=True,inplace=True)
    return intraday_data

In [8]:
for index, row in path.iterrows():
    try:
        data = pd.read_pickle(row['location'])
        entry_datetime=datetime.datetime.combine(row['datadate'].date(),datetime.time(10,30))
        exit_datetime=datetime.datetime.combine(row['datadate'].date(),datetime.time(15,20))
        #bnf future & atm
        data['expiry_type']=np.where((data['instrument_type']=='FUT'),(data['ticker'].apply(lambda x:x.split('-')[-1].split('.')[0])),"")
        current_month_exp= 'I' # I-current month expiry, II-next month expiry, III- far month expiry
        instrument='BANKNIFTY'
        future_data=data[(data['instrument_type']=='FUT') & (data['instrument_name']==instrument) & (data['expiry_type']==current_month_exp)]
        future_data.reset_index(drop=True,inplace=True)
        atm=future_data[future_data['datetime']==entry_datetime]['open'].iloc[0]
        base_price=100
        atm=base_price*round(atm/base_price)
        #print(atm)
        itm = atm - 0.02*atm
        itm = base_price*round(itm/base_price)
        #print(itm)
        otm = atm + 0.02*atm
        otm = base_price*round(otm/base_price)
        #print(otm)

        # CE data & PE data for atm
        nearest_expiry=row['datadate'].date()+relativedelta(weekday=TH)
        atm_ce_data=data[(data['instrument_type']=='CE') & (data['instrument_name']==instrument) & (data['strike_price']==atm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        atm_ce_data.reset_index(drop=True,inplace=True)
        atm_pe_data=data[(data['instrument_type']=='PE') & (data['instrument_name']==instrument) & (data['strike_price']==atm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        atm_pe_data.reset_index(drop=True,inplace=True)
        #print(atm_ce_data)
        # CE data & PE data for itm
        itm_ce_data=data[(data['instrument_type']=='CE') & (data['instrument_name']==instrument) & (data['strike_price']==itm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        itm_ce_data.reset_index(drop=True,inplace=True)
        itm_pe_data=data[(data['instrument_type']=='PE') & (data['instrument_name']==instrument) & (data['strike_price']==itm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        itm_pe_data.reset_index(drop=True,inplace=True)
        #print(itm_ce_data)
        # CE data & PE data for otm
        otm_ce_data=data[(data['instrument_type']=='CE') & (data['instrument_name']==instrument) & (data['strike_price']==otm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        otm_ce_data.reset_index(drop=True,inplace=True)
        otm_pe_data=data[(data['instrument_type']=='PE') & (data['instrument_name']==instrument) & (data['strike_price']==otm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        otm_pe_data.reset_index(drop=True,inplace=True)

        #symbol
        atm_ce_symbol=atm_ce_data['ticker'].iloc[0]
        atm_pe_symbol=atm_pe_data['ticker'].iloc[0]
        itm_ce_symbol=itm_ce_data['ticker'].iloc[0]
        itm_pe_symbol=itm_pe_data['ticker'].iloc[0]
        otm_ce_symbol=otm_ce_data['ticker'].iloc[0]
        otm_pe_symbol=otm_pe_data['ticker'].iloc[0]
        atm_intraday_data = data_pre(future_data,atm_ce_data,atm_pe_data)
        itm_intraday_data = data_pre(future_data,itm_ce_data,itm_pe_data)
        otm_intraday_data = data_pre(future_data,otm_ce_data,otm_pe_data)

        #Entry price
        future_price= future_data[future_data['datetime']==entry_datetime]['open'].iloc[0]
        atm_ce_entry_price= atm_intraday_data[atm_intraday_data['datetime']==entry_datetime]['ce_close'].iloc[0]
        atm_pe_entry_price= atm_intraday_data[atm_intraday_data['datetime']==entry_datetime]['pe_close'].iloc[0]
        itm_ce_entry_price= itm_intraday_data[itm_intraday_data['datetime']==entry_datetime]['ce_close'].iloc[0]
        itm_pe_entry_price= itm_intraday_data[itm_intraday_data['datetime']==entry_datetime]['pe_close'].iloc[0]
        otm_ce_entry_price= otm_intraday_data[otm_intraday_data['datetime']==entry_datetime]['ce_close'].iloc[0]
        otm_pe_entry_price= otm_intraday_data[otm_intraday_data['datetime']==entry_datetime]['pe_close'].iloc[0]

        print(row['datadate'])
        print("ATM: ",atm)
        trade_log = sell(atm_intraday_data,trade_log,atm,atm_ce_entry_price,atm_pe_entry_price,atm_ce_symbol,atm_pe_symbol)
        print("ITM: ",itm)
        trade_log = buy(itm_intraday_data,trade_log,itm,itm_ce_entry_price,itm_pe_entry_price,itm_ce_symbol,itm_pe_symbol)
        print("OTM: ",otm)
        trade_log = buy(otm_intraday_data,trade_log,otm,otm_ce_entry_price,otm_pe_entry_price,otm_ce_symbol,otm_pe_symbol)
    except:
        pass

2019-01-01 00:00:00
ATM:  27200
CE SL hit
PE exit at 15:10
ITM:  26700
PE SL hit
CE exit at 15:10
OTM:  27700
CE SL hit
PE SL hit
2019-01-02 00:00:00
ATM:  27500
PE SL hit
CE exit at 15:10
ITM:  27000
CE SL hit
PE exit at 15:10
OTM:  28000
CE SL hit
PE exit at 15:10
2019-01-03 00:00:00
ATM:  27300
PE SL hit
CE exit at 15:10
ITM:  26800
CE SL hit
PE SL hit
OTM:  27800
CE SL hit
PE exit at 15:10
2019-01-04 00:00:00
ATM:  27100
CE SL hit
PE exit at 15:10
ITM:  26600
PE SL hit
CE exit at 15:10
OTM:  27600
PE SL hit
CE exit at 15:10
2019-01-08 00:00:00
ATM:  27400
PE SL hit
CE SL hit
ITM:  26900
PE SL hit
CE exit at 15:10
OTM:  27900
CE SL hit
PE SL hit
2019-01-09 00:00:00
ATM:  27700
CE SL hit
PE SL hit
ITM:  27100
PE SL hit
CE SL hit
OTM:  28300
CE SL hit
PE exit at 15:10
2019-01-10 00:00:00
ATM:  27700
PE SL hit
CE exit at 15:10
ITM:  27100
PE SL hit
CE SL hit
OTM:  28300
CE SL hit
PE exit at 15:10
2019-01-11 00:00:00
ATM:  27600
PE SL hit
CE exit at 15:10
ITM:  27000
CE & PE did not hit

In [9]:
trade_log

,Entry_date_time,Future_price,Transaction_Type,Strike,Type_of_Strike,Days_to_expiry,CE_symbol,CE_entry_price,CE_exit_price,CE_exit_datetime,CE_pnl,PE_symbol,PE_entry_price,PE_exit_price,PE_exit_datetime,PE_pnl,Total_pnl
0,2019-01-01 10:30:00,27198.90,Sell,27200,ATM,2,BANKNIFTY03JAN1927200CE.NFO,106.95,128.34,2019-01-01 14:29:00,-21.39,BANKNIFTY03JAN1927200PE.NFO,199.00,54.30,2019-01-01 15:20:00,144.70,123.31
1,2019-01-01 10:30:00,27198.90,Buy,26700,ITM,2,BANKNIFTY03JAN1926700CE.NFO,420.05,689.20,2019-01-01 15:20:00,269.15,BANKNIFTY03JAN1926700PE.NFO,28.80,22.95,2019-01-01 12:05:00,-5.85,263.30
2,2019-01-01 10:30:00,27198.90,Buy,27700,OTM,2,BANKNIFTY03JAN1927700CE.NFO,8.20,6.55,2019-01-01 13:33:00,-1.65,BANKNIFTY03JAN1927700PE.NFO,612.70,490.16,2019-01-01 14:29:00,-122.54,-124.19
3,2019-01-02 10:30:00,27487.65,Sell,27500,ATM,1,BANKNIFTY03JAN1927500CE.NFO,74.15,20.70,2019-01-02 15:20:00,53.45,BANKNIFTY03JAN1927500PE.NFO,185.75,222.90,2019-01-02 12:21:00,-37.15,16.30
4,2019-01-02 10:30:00,27487.65,Buy,27000,ITM,1,BANKNIFTY03JAN1927000CE.NFO,392.80,310.05,2019-01-02 12:22:00,-82.75,BANKNIFTY03JAN1927000PE.NFO,24.55,48.55,2019-01-02 15:20:00,24.00,-58.75
5,2019-01-02 10:30:00,27487.65,Buy,28000,OTM,1,BANKNIFTY03JAN1928000CE.NFO,3.50,2.65,2019-01-02 11:28:00,-0.85,BANKNIFTY03JAN1928000PE.NFO,605.50,824.00,2019-01-02 15:20:00,218.50,217.65
6,2019-01-03 10:30:00,27271.00,Sell,27300,ATM,0,BANKNIFTY03JAN1927300CE.NFO,26.15,0.05,2019-01-03 15:20:00,26.10,BANKNIFTY03JAN1927300PE.NFO,157.65,189.18,2019-01-03 10:39:00,-31.53,-5.43
7,2019-01-03 10:30:00,27271.00,Buy,26800,ITM,0,BANKNIFTY03JAN1926800CE.NFO,346.35,270.40,2019-01-03 10:45:00,-75.95,BANKNIFTY03JAN1926800PE.NFO,2.85,2.28,2019-01-03 13:08:00,-0.57,-76.52
8,2019-01-03 10:30:00,27271.00,Buy,27800,OTM,0,BANKNIFTY03JAN1927800CE.NFO,0.10,0.05,2019-01-03 10:52:00,-0.05,BANKNIFTY03JAN1927800PE.NFO,630.20,804.05,2019-01-03 15:20:00,173.85,173.80
9,2019-01-04 10:30:00,27099.85,Sell,27100,ATM,6,BANKNIFTY10JAN1927100CE.NFO,177.70,213.24,2019-01-04 11:00:00,-35.54,BANKNIFTY10JAN1927100PE.NFO,270.50,136.10,2019-01-04 15:20:00,134.40,98.86


In [13]:
trade_log.to_csv("Trade_log.csv",index=False)